In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data = pd.read_excel('dataset/전국스마트가로등표준데이터-20210801.xls',index_col=0)
data.columns = data.iloc[0]
data = data[1:]
seoul_data = data.query('시도명=="서울특별시"')

seoul_data.drop(['소재지도로명주소','도로명','제공기관코드','제공기관명','관리기관전화번호','시도명','앱서비스명','앱서비스제공내용','관리기관명'],axis=1,inplace=True)
seoul_data.센싱정보서비스.fillna('없음',inplace=True)
seoul_data.기타서비스.fillna('없음',inplace=True)
seoul_data.index = seoul_data.스마트가로등형태
seoul_data.drop('스마트가로등형태',axis=1,inplace=True)
seoul_data.head()

스마트가로등유형,시군구명,소재지지번주소,위도,경도,점등상태여부,센서종류,센싱정보서비스,CCTV유무,WiFi유무,GPS수신기유무,비콘유무,조명제어여부,위급상황신고가능여부,기타서비스,설치연도,데이터기준일자
스마트가로등형태,,,,,,,,,,,,,,,,
01,강서구,서울특별시 강서구 공항동 2-104,37.562692,NaN,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,NaN,NaN,Y,전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,37.562441,126.816405,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,37.562468,126.816555,Y,전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,37.562164,126.816405,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19


In [2]:
LY_null = seoul_data[(seoul_data['위도'].isnull()==1) | (seoul_data['경도'].isnull()==1)]

In [3]:
LY_null.head(1)

스마트가로등유형,시군구명,소재지지번주소,위도,경도,점등상태여부,센서종류,센싱정보서비스,CCTV유무,WiFi유무,GPS수신기유무,비콘유무,조명제어여부,위급상황신고가능여부,기타서비스,설치연도,데이터기준일자
스마트가로등형태,,,,,,,,,,,,,,,,
01,강서구,서울특별시 강서구 공항동 2-104,37.562692,NaN,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19


In [4]:
add_list = LY_null['소재지지번주소'].unique().tolist()

In [5]:
location = '서울특별시 강서구 공항동 2-104'

In [6]:
key = '228A382D-10C5-3241-9A08-F8D0F29726EE'
version = '2.0'
crs='EPSG:4326'
address = location
refine= 'true'
simple ='false'
format_ = 'json'
type_ = 'parcel'

In [7]:
url = f'http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&version={version}&crs={crs}&address={address}&refine={refine}&simple={simple}&format={format_}&type={type_}'

In [8]:
import requests
response = requests.get(url)
data = response.json()

In [9]:
data['response']['result']['point']['x']

'126.816476653663'

In [10]:
def get_xy(x):
    key = '228A382D-10C5-3241-9A08-F8D0F29726EE'
    version = '2.0'
    crs='EPSG:4326'
    address = x
    refine= 'true'
    simple ='false'
    format_ = 'json'
    type_ = 'parcel'
    url = f'http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&version={version}&crs={crs}&address={address}&refine={refine}&simple={simple}&format={format_}&type={type_}'
    response = requests.get(url)
    data = response.json()
    y = data['response']['result']['point']['x']
    x = data['response']['result']['point']['y']
    return x,y

In [11]:
for i in add_list:
    LY_null.loc[LY_null['소재지지번주소']==i,'위도'] = get_xy(i)[0]
    LY_null.loc[LY_null['소재지지번주소']==i,'경도'] = get_xy(i)[1]

In [12]:
seoul_data.isnull().sum()

스마트가로등유형
시군구명           0
소재지지번주소        0
위도            19
경도            41
점등상태여부         0
센서종류           0
센싱정보서비스        0
CCTV유무         0
WiFi유무         0
GPS수신기유무       0
비콘유무           0
조명제어여부         0
위급상황신고가능여부     0
기타서비스          0
설치연도           0
데이터기준일자        0
dtype: int64

In [13]:
import numpy as np

In [14]:
seoul_data.columns

Index(['시군구명', '소재지지번주소', '위도', '경도', '점등상태여부', '센서종류', '센싱정보서비스', 'CCTV유무',
       'WiFi유무', 'GPS수신기유무', '비콘유무', '조명제어여부', '위급상황신고가능여부', '기타서비스', '설치연도',
       '데이터기준일자'],
      dtype='object', name='스마트가로등유형')

In [15]:
LY_null.columns

Index(['시군구명', '소재지지번주소', '위도', '경도', '점등상태여부', '센서종류', '센싱정보서비스', 'CCTV유무',
       'WiFi유무', 'GPS수신기유무', '비콘유무', '조명제어여부', '위급상황신고가능여부', '기타서비스', '설치연도',
       '데이터기준일자'],
      dtype='object', name='스마트가로등유형')

In [16]:
seoul_data[(seoul_data['위도'].isnull()==1) | (seoul_data['경도'].isnull()==1)] = LY_null

In [17]:
seoul_data.isnull().sum()

스마트가로등유형
시군구명          0
소재지지번주소       0
위도            0
경도            0
점등상태여부        0
센서종류          0
센싱정보서비스       0
CCTV유무        0
WiFi유무        0
GPS수신기유무      0
비콘유무          0
조명제어여부        0
위급상황신고가능여부    0
기타서비스         0
설치연도          0
데이터기준일자       0
dtype: int64

In [18]:
seoul_data

스마트가로등유형,시군구명,소재지지번주소,위도,경도,점등상태여부,센서종류,센싱정보서비스,CCTV유무,WiFi유무,GPS수신기유무,비콘유무,조명제어여부,위급상황신고가능여부,기타서비스,설치연도,데이터기준일자
스마트가로등형태,,,,,,,,,,,,,,,,
01,강서구,서울특별시 강서구 공항동 2-104,37.5620174418088,126.816476653663,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,37.5620174418088,126.816476653663,Y,전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,37.562441,126.816405,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,37.562468,126.816555,Y,전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 공항동 2-104,37.562164,126.816405,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01,강서구,서울특별시 강서구 방화동 217-22,37.5683429406593,126.816670265057,Y,전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 방화동 217-22,37.5683429406593,126.816670265057,Y,속도센서+전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19
01,강서구,서울특별시 강서구 방화동 217-22,37.5683429406593,126.816670265057,Y,전류센서,도로조명,N,N,N,N,Y,N,전력사용량정보+고장정보,2020,2021-03-19


In [19]:
seoul_data.to_csv('서울스마트가로등표준데이터셋.csv')